In [8]:
# importing libraries
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [22]:
# read dataset from csv file
df <- read.csv("../data/Dataset- Superstore (2015-2018).csv")
head(df)

ï..Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Country,City,...,Postal.Code,Region,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016/11/08,2016/11/11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2016-152156,2016/11/08,2016/11/11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.9400,3,0.00,219.5820
3,CA-2016-138688,2016/06/12,2016/06/16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.6200,2,0.00,6.8714
4,US-2015-108966,2015/10/11,2015/10/18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2015-108966,2015/10/11,2015/10/18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
6,CA-2014-115812,2014/06/09,2014/06/14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.8600,7,0.00,14.1694


In [23]:
# finding the number of rows and columns
dim(df)

[1] 9994   21

In [15]:
# dataset information
str(df)

'data.frame':	9994 obs. of  21 variables:
 $ ï..Row.ID    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Order.ID     : Factor w/ 5009 levels "CA-2014-100006",..: 2501 2501 2297 4373 4373 202 202 202 202 202 ...
 $ Order.Date   : Factor w/ 1237 levels "2014/01/03","2014/01/04",..: 865 865 733 520 520 126 126 126 126 126 ...
 $ Ship.Date    : Factor w/ 1334 levels "2014/01/07","2014/01/08",..: 930 930 788 569 569 129 129 129 129 129 ...
 $ Ship.Mode    : Factor w/ 4 levels "First Class",..: 3 3 3 4 4 4 4 4 4 4 ...
 $ Customer.ID  : Factor w/ 793 levels "AA-10315","AA-10375",..: 144 144 240 706 706 89 89 89 89 89 ...
 $ Customer.Name: Factor w/ 793 levels "Aaron Bergman",..: 167 167 202 688 688 114 114 114 114 114 ...
 $ Segment      : Factor w/ 3 levels "Consumer","Corporate",..: 1 1 2 1 1 1 1 1 1 1 ...
 $ Country      : Factor w/ 1 level "United States": 1 1 1 1 1 1 1 1 1 1 ...
 $ City         : Factor w/ 531 levels "Aberdeen","Abilene",..: 195 195 267 154 154 267 267 267 267 267 ...
 $ State     

In [17]:
# dataset summary
summary(df)

   ï..Row.ID              Order.ID         Order.Date        Ship.Date   
 Min.   :   1   CA-2017-100111:  14   2016/09/05:  38   2015/12/16:  35  
 1st Qu.:2499   CA-2017-157987:  12   2017/09/02:  36   2017/09/26:  34  
 Median :4998   CA-2016-165330:  11   2016/11/10:  35   2017/11/21:  32  
 Mean   :4998   US-2016-108504:  11   2017/12/01:  34   2017/12/06:  32  
 3rd Qu.:7496   CA-2015-131338:  10   2017/12/02:  34   2017/09/06:  30  
 Max.   :9994   CA-2016-105732:  10   2017/12/09:  33   2017/09/15:  30  
                (Other)       :9926   (Other)   :9784   (Other)   :9801  
          Ship.Mode      Customer.ID               Customer.Name 
 First Class   :1538   WB-21850:  37   William Brown      :  37  
 Same Day      : 543   JL-15835:  34   John Lee           :  34  
 Second Class  :1945   MA-17560:  34   Matt Abelman       :  34  
 Standard Class:5968   PP-18955:  34   Paul Prost         :  34  
                       CK-12205:  32   Chloris Kastensmidt:  32  
            

In [26]:
# occurences of each datatype
table(sapply(df, class))


 factor integer numeric 
     15       3       3 

In [19]:
# finding the number of unique values in each column
sapply(df, function(x) length(unique(x)))

ï..Row.ID      Order.ID    Order.Date     Ship.Date     Ship.Mode 
         9994          5009          1237          1334             4 
  Customer.ID Customer.Name       Segment       Country          City 
          793           793             3             1           531 
        State   Postal.Code        Region    Product.ID      Category 
           49           631             4          1862             3 
 Sub.Category  Product.Name         Sales      Quantity      Discount 
           17          1850          5825            14            12 
       Profit 
         7287

In [32]:
# finding the number of missing values in each column
sapply(df, function(x) sum(is.na(x)))

ï..Row.ID      Order.ID    Order.Date     Ship.Date     Ship.Mode 
            0             0             0             0             0 
  Customer.ID Customer.Name       Segment       Country          City 
            0             0             0             0             0 
        State   Postal.Code        Region    Product.ID      Category 
            0             0             0             0             0 
 Sub.Category  Product.Name         Sales      Quantity      Discount 
            0             0             0             0             0 
       Profit 
            0

In [38]:
 cat("Total of missing values:", sum(is.na(df)))

Total of missing values: 0